In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# import plotly_express as px
import pandas_profiling

#lines below let allow multiple results from a line of code to be shown e.g. df.head() + df.columns
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#this allows us to see all of our columns or rows in jupyter notebook
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

#filter future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#set our code up so that it doesn't display scientific notation, we want full numbers
pd.set_option('display.float_format', '{:.2f}'.format)

__Had to import the training and test set independent of each other since my sad computer is only 8GB RAM, It crashed importing them both at the same time__

In [34]:
#import data
train_identity = pd.read_csv('train_identity.csv')
train_transaction = pd.read_csv('train_transaction.csv')
# test_identity = pd.read_csv('test_identity.csv')
# test_transaction = pd.read_csv('test_transaction.csv')
#merge data
train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
# test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)

In [6]:
train.shape

(590540, 435)

In [35]:
test_identity = pd.read_csv('test_identity.csv')
test_transaction = pd.read_csv('test_transaction.csv')
test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)

In [8]:
test.shape

(506691, 434)

In [7]:
train.profile_report()

In [30]:
# Printing the percentage of missing values per column
def percent_missing(dataframe):
    '''
    Prints the percentage of missing values for each column in a dataframe
    '''
    # Summing the number of missing values per column and then dividing by the total
    sumMissing = dataframe.isnull().values.sum(axis=0)
    pctMissing = sumMissing / dataframe.shape[0]
    
    if sumMissing.sum() == 0:
        print('No missing values')
    else:
        # Looping through and printing out each columns missing value percentage
        print('Percent Missing Values:', '\n')
        for idx, col in enumerate(dataframe.columns):
            if sumMissing[idx] > 0:
                print('{0}: {1:.2f}%'.format(col, pctMissing[idx] * 100))

In [10]:
#https://stackoverflow.com/questions/29294983/how-to-calculate-correlation-between-all-columns-and-remove-highly-correlated-on
def filter_df_corr(inp_data, corr_val):
    '''
    Returns an array or dataframe (based on type(inp_data) adjusted to drop \
        columns with high correlation to one another. Takes second arg corr_val
        that defines the cutoff

    ----------
    inp_data : np.array, pd.DataFrame
        Values to consider
    corr_val : float
        Value [0, 1] on which to base the correlation cutoff
    '''
# Creates Correlation Matrix
    if isinstance(inp_data, np.ndarray):
        inp_data = pd.DataFrame(data=inp_data)
        array_flag = True
    else:
        array_flag = False
    corr_matrix = inp_data.corr()

    # Iterates through Correlation Matrix Table to find correlated columns
    drop_cols = []
    n_cols = len(corr_matrix.columns)

    for i in range(n_cols):
        for k in range(i+1, n_cols):
            val = corr_matrix.iloc[k, i]
            col = corr_matrix.columns[i]
            row = corr_matrix.index[k]
            if abs(val) >= corr_val:
                # Prints the correlated feature set and the corr val
                print(col, "|", row, "|", round(val, 2))
                drop_cols.append(col)

    # Drops the correlated columns
    drop_cols = set(drop_cols)
    inp_data = inp_data.drop(columns=drop_cols)
    # Return same type as inp
    if array_flag:
        return inp_data.values
    else:
        return inp_data

In [39]:
# #drop columns missing more than half their data
# for column in train:
#     if train[column].isna().sum() > 50000:
#         train.drop(column, axis = 1, inplace = True)

__Mean Target Encoding Categorical Features__

In [49]:
# card_means = train.groupby('card4').isFraud.mean()
# train['card4'] = train['card4'].map(card_means)
# train['card4'].unique()

# #remove NA values
# train.dropna(subset = ['card4'], inplace = True)

array([0.0253078 , 0.02881854, 0.02486205, 0.01141141,        nan])

# Baseline Modeling

In [36]:
train.fillna(-1234, inplace = True)
test.fillna(-1234, inplace = True)

In [328]:
label = train['isFraud']
features = train.drop(['isFraud'], axis = 1)
    # #Label Encoding
for x in features.columns[features.dtypes == 'object']:
    features[x] = features[x].factorize()[0]
# X_train, X_test, y_train, y_test = ms.train_test_split(features, label, test_size = .2, random_state = 46)

In [38]:
import xgboost as xgb
model = xgb.XGBClassifier(n_estimators=500,max_depth=9,learning_rate=0.05,subsample=0.9, colsample_bytree=0.9, missing = -1234)
model.fit(features, label)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=-1234, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.9, verbosity=1)

In [326]:
# from sklearn.metrics import classification_report
# X_test_predictions = lm.predict(X_test)
# print(classification_report(y_test, X_test_predictions))

In [40]:
for x in test.columns[test.dtypes == 'object']:
    test[x] = test[x].factorize()[0]

__process of breaking the test up into pieces to predict :) gotta love 8GB RAM__

In [258]:
test1 = test.iloc[0:200000, :]

In [313]:
test2 = test.iloc[200000:400000, :]

In [314]:
test3 = test.iloc[400000:506691, :]

In [259]:
pred = model.predict(test1)

In [319]:
pred2 = model.predict(test2)

In [321]:
pred3 = model.predict(test3)

In [323]:
p1 = pd.DataFrame(pred)
p2 = pd.DataFrame(pred2)
p3 = pd.DataFrame(pred3)
predictions_df = pd.concat([p1, p2, p3])

In [325]:
predictions_df.to_csv('predictions.csv')

# Scored a .73 on the leaderboard with this baseline
- TODO: tune model hyperparameters
- drop highly correlated features
- explore timeseries options?